In [5]:
import os
from transformers import AutoTokenizer, MBartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import wandb
from usecrets import WANDB_API_KEY

os.environ["WANDB_API_KEY"] = WANDB_API_KEY
wandb.login()



num_epochs = 10
batch_size = 32
warmup_steps = 2000
weight_decay = 0.01

wandb.init(
    project="coursework_2025",
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "warmup_steps": warmup_steps,
        "weight_decay": weight_decay
    }
)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
dataset = load_dataset("json", data_files="train.jsonl", field=None)

tokenizer = AutoTokenizer.from_pretrained('./ru-mbart-large-summ')
model = MBartForConditionalGeneration.from_pretrained('./ru-mbart-large-summ')
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

def preprocess_function(examples):
    inputs = examples["text"]
    targets = examples["summary"]
    
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding=True)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

training_args = TrainingArguments(
    output_dir="./tuned",     
    num_train_epochs=num_epochs,                      
    per_device_train_batch_size=batch_size,          
    per_device_eval_batch_size=batch_size,            
    warmup_steps=warmup_steps,                       
    weight_decay=weight_decay,                       
    logging_dir="./logs",                    
    logging_steps=50,                        
    evaluation_strategy="steps",             
    eval_steps=2000,                         
    save_total_limit=5,                      
    save_steps=2000,
    report_to=["wandb"]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    tokenizer=tokenizer,
)

wandb.watch(model, log="all")

trainer.train()

model.save_pretrained(os.path.join(training_args.output_dir, "final_model"))
tokenizer.save_pretrained(os.path.join(training_args.output_dir, "final_model"))

wandb.finish()

/opt/homebrew/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/72 [00:00<?, ?it/s]

{'loss': 3.022, 'grad_norm': 4.973212718963623, 'learning_rate': 5e-06, 'epoch': 2.08}


/opt/homebrew/lib/python3.12/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'train_runtime': 96.0231, 'train_samples_per_second': 1.468, 'train_steps_per_second': 0.75, 'train_loss': 2.9450414975484214, 'epoch': 3.0}


('./tuned/final_model/tokenizer_config.json',
 './tuned/final_model/special_tokens_map.json',
 './tuned/final_model/sentencepiece.bpe.model',
 './tuned/final_model/added_tokens.json',
 './tuned/final_model/tokenizer.json')

In [ ]:
import os
import torch
from transformers import AutoTokenizer, MBartForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
from bert_score import score as bert_score_metric 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

raw_dataset = load_dataset("json", data_files="train.jsonl", field=None)["train"]

split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=52)
train_raw = split_dataset["train"]
val_raw = split_dataset["test"]

tokenizer = AutoTokenizer.from_pretrained('./ru-mbart-large-summ')
model = MBartForConditionalGeneration.from_pretrained('./ru-mbart-large-summ')
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

max_input_length = 1024
max_target_length = 128

def chunk_preprocess(example):
    text = example["text"]
    summary = example["summary"]
    
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    target_ids = tokenizer.encode(summary, add_special_tokens=True)
    
    if len(input_ids) > max_input_length:
        chunk_examples = []
        for i in range(0, len(input_ids), max_input_length):
            chunk_ids = input_ids[i:i+max_input_length]
            # Паддинг для чанка до max_input_length
            padded_chunk = chunk_ids + [tokenizer.pad_token_id] * (max_input_length - len(chunk_ids))
            attention_mask = [1] * len(chunk_ids) + [0] * (max_input_length - len(chunk_ids))
            chunk_examples.append({
                "input_ids": padded_chunk,
                "attention_mask": attention_mask,
                "labels": target_ids,  # Для каждого чанка используется та же сводка
            })
        return chunk_examples
    else:
        encoded_inputs = tokenizer(text, max_length=max_input_length, truncation=True, padding="max_length")
        with tokenizer.as_target_tokenizer():
            encoded_targets = tokenizer(summary, max_length=max_target_length, truncation=True, padding="max_length")
        encoded_inputs["labels"] = encoded_targets["input_ids"]
        return encoded_inputs

train_dataset = train_raw.flat_map(chunk_preprocess)
val_dataset = val_raw.flat_map(chunk_preprocess)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        ce_loss = outputs.loss

        generated_tokens = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_target_length,
            num_beams=4,
            early_stopping=True
        )

        preds = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        targets = self.tokenizer.batch_decode(inputs["labels"], skip_special_tokens=True)

        P, R, F1 = bert_score_metric(preds, targets, lang="ru", verbose=False)
        F1_tensor = torch.tensor(F1, device=ce_loss.device)
        # Дополнительный лосс: (1 - F1), так как оптимизатор минимизирует loss
        bert_loss = (1 - F1_tensor).mean()

        alpha = 0.5
        loss = ce_loss + alpha * bert_loss
        loss = bert_loss

        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./tuned",     
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch", 
    save_total_limit=2,
    save_strategy="epoch",
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()

model.save_pretrained(os.path.join(training_args.output_dir, "final_model"))
tokenizer.save_pretrained(os.path.join(training_args.output_dir, "final_model"))


In [10]:
import os
import torch
from transformers import AutoTokenizer, MBartForConditionalGeneration, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset, Dataset
from bert_score import score as bert_score_metric
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

raw_dataset = load_dataset("json", data_files="train.jsonl", field=None)["train"]
split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=52)
train_raw = split_dataset["train"]
val_raw = split_dataset["test"]

tokenizer = AutoTokenizer.from_pretrained('./ru-mbart-large-summ')
model = MBartForConditionalGeneration.from_pretrained('./ru-mbart-large-summ').to(device)
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

max_input_length = 1024
max_target_length = 128

def chunk_preprocess(example):
    text = example["text"]
    summary = example["summary"]
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    target_ids = tokenizer.encode(summary, add_special_tokens=True)
    if len(input_ids) > max_input_length:
        chunk_examples = []
        for i in range(0, len(input_ids), max_input_length):
            chunk_ids = input_ids[i:i+max_input_length]
            padded_chunk = chunk_ids + [tokenizer.pad_token_id] * (max_input_length - len(chunk_ids))
            attention_mask = [1] * len(chunk_ids) + [0] * (max_input_length - len(chunk_ids))
            padded_target_ids = target_ids[:max_target_length]
            if len(padded_target_ids) < max_target_length:
                padded_target_ids += [tokenizer.pad_token_id] * (max_target_length - len(padded_target_ids))
            chunk_examples.append({
                "input_ids": padded_chunk,
                "attention_mask": attention_mask,
                "labels": padded_target_ids,
            })
        return chunk_examples
    else:
        encoded_inputs = tokenizer(text, max_length=max_input_length, truncation=True, padding="max_length")
        with tokenizer.as_target_tokenizer():
            encoded_targets = tokenizer(summary, max_length=max_target_length, truncation=True, padding="max_length")
        encoded_inputs["labels"] = encoded_targets["input_ids"]
        return encoded_inputs

def process_dataset(dataset):
    examples = []
    for example in dataset:
        processed = chunk_preprocess(example)
        if isinstance(processed, list):
            examples.extend(processed)
        else:
            examples.append(processed)
    return examples

train_examples = process_dataset(train_raw)
val_examples = process_dataset(val_raw)
train_dataset = Dataset.from_list(train_examples)
val_dataset = Dataset.from_list(val_examples)

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        ce_loss = outputs.loss
        generated_tokens = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=max_target_length,
            num_beams=4,
            early_stopping=True
        )
        preds = self.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        targets = self.tokenizer.batch_decode(inputs["labels"], skip_special_tokens=True)
        P, R, F1 = bert_score_metric(preds, targets, lang="ru", model_type="./bert-base-multilingual-cased", num_layers=9, verbose=False)
        F1_tensor = torch.tensor(F1, device=ce_loss.device)
        alpha = 0.5
        loss = ce_loss + alpha * (1 - F1_tensor).mean()
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="./tuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_total_limit=2,
    save_strategy="epoch",
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

trainer.train()
model.save_pretrained(os.path.join(training_args.output_dir, "final_model"))
tokenizer.save_pretrained(os.path.join(training_args.output_dir, "final_model"))


Используется устройство: cpu


Token indices sequence length is longer than the specified maximum sequence length for this model (1233 > 1024). Running this sequence through the model will result in indexing errors


  0%|          | 0/10110 [00:00<?, ?it/s]

/var/folders/8n/y27w9hnn0q5dqzq6m21wkz_w0000gn/T/ipykernel_12322/971686660.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  F1_tensor = torch.tensor(F1, device=ce_loss.device)


KeyboardInterrupt: 

In [ ]:
import os
import torch
import wandb

from transformers import (
    AutoTokenizer,
    MBartForConditionalGeneration,
    Trainer,
    TrainingArguments,
    TrainerCallback
)
from datasets import load_dataset, Dataset
from bert_score import score as bert_score_metric


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")
if device.type == "cuda":
    print("GPU Device Name:", torch.cuda.get_device_name(0))


num_epochs = 10
batch_size = 32
weight_decay = 0.01


wandb.init(
    project="coursework_2025",
    config={
        "num_train_epochs": num_epochs,
        "batch_size": batch_size,
        "warmup_steps": 500,
        "weight_decay": weight_decay,
    }
)

raw_dataset = load_dataset("json", data_files="train.jsonl", field=None)["train"]
split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=52)
train_raw = split_dataset["train"]
val_raw = split_dataset["test"]


tokenizer = AutoTokenizer.from_pretrained("d0rj/ru-mbart-large-summ")
model = MBartForConditionalGeneration.from_pretrained("d0rj/ru-mbart-large-summ").to(device)

tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "ru_RU"

max_input_length = 1024
max_target_length = 128


def chunk_preprocess(example):
    text = example["text"]
    summary = example["summary"]
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    target_ids = tokenizer.encode(summary, add_special_tokens=True)
    
    if len(input_ids) > max_input_length:
        chunk_examples = []
        for i in range(0, len(input_ids), max_input_length):
            chunk_ids = input_ids[i:i+max_input_length]
            padded_chunk = chunk_ids + [tokenizer.pad_token_id] * (max_input_length - len(chunk_ids))
            attention_mask = [1] * len(chunk_ids) + [0] * (max_input_length - len(chunk_ids))

            padded_target_ids = target_ids[:max_target_length]
            if len(padded_target_ids) < max_target_length:
                padded_target_ids += [tokenizer.pad_token_id] * (max_target_length - len(padded_target_ids))
            
            chunk_examples.append({
                "input_ids": padded_chunk,
                "attention_mask": attention_mask,
                "labels": padded_target_ids,
            })
        return chunk_examples
    else:
        encoded_inputs = tokenizer(
            text, max_length=max_input_length, truncation=True, padding="max_length"
        )
        with tokenizer.as_target_tokenizer():
            encoded_targets = tokenizer(
                summary, max_length=max_target_length, truncation=True, padding="max_length"
            )
        encoded_inputs["labels"] = encoded_targets["input_ids"]
        return encoded_inputs

def process_dataset(dataset):
    examples = []
    for example in dataset:
        processed = chunk_preprocess(example)
        if isinstance(processed, list):
            examples.extend(processed)
        else:
            examples.append(processed)
    return examples

train_examples = process_dataset(train_raw)
val_examples = process_dataset(val_raw)

train_dataset = Dataset.from_list(train_examples)
val_dataset = Dataset.from_list(val_examples)

class CustomTrainer(Trainer):
    """
    Переопределяем только compute_loss, чтобы использовать
    стандартную кросс-энтропию (outputs.loss).
    """
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        ce_loss = outputs.loss
        return (ce_loss, outputs) if return_outputs else ce_loss


class MetricsLoggerCallback(TrainerCallback):
    def __init__(self, log_file="training_log.txt", max_eval_samples=200):
        """
        max_eval_samples: сколько максимально примеров брать
        из валидационного набора для вычисления BERTScore.
        Увеличение этого числа делает вычисление точнее, но медленнее.
        """
        super().__init__()
        self.log_file = log_file
        self.max_eval_samples = max_eval_samples

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """
        Вызывается после завершения валидации в конце каждой эпохи.
        """
        if metrics is None:
            metrics = {}
        
        trainer = kwargs["trainer"]
        model = trainer.model
        tokenizer = trainer.tokenizer
        eval_dataset = trainer.eval_dataset
        
        subset_size = min(len(eval_dataset), self.max_eval_samples)
        small_eval = eval_dataset.select(range(subset_size))
        
        all_preds = []
        all_labels = []
        
        for start_idx in range(0, subset_size, args.per_device_eval_batch_size):
            batch = small_eval[start_idx : start_idx + args.per_device_eval_batch_size]
            
            # Преобразуем list of dict -> тензоры
            input_ids = torch.tensor([x["input_ids"] for x in batch], dtype=torch.long).to(model.device)
            attention_mask = torch.tensor([x["attention_mask"] for x in batch], dtype=torch.long).to(model.device)
            labels = torch.tensor([x["labels"] for x in batch], dtype=torch.long).to(model.device)

            # Генерация
            with torch.no_grad():
                generated_tokens = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_length=max_target_length,
                    num_beams=4,
                    early_stopping=True
                )

            # Декодируем
            preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            tgts = tokenizer.batch_decode(labels, skip_special_tokens=True)

            all_preds.extend(preds)
            all_labels.extend(tgts)

        P, R, F1 = bert_score_metric(
            all_preds, 
            all_labels, 
            lang="ru",
            model_type="google-bert/bert-base-multilingual-cased",
            num_layers=9,
            verbose=False
        )

        p_mean = float(torch.mean(P))
        r_mean = float(torch.mean(R))
        f1_mean = float(torch.mean(F1))

        metrics["bert_score_precision"] = p_mean
        metrics["bert_score_recall"] = r_mean
        metrics["bert_score_f1"] = f1_mean

        wandb.log(metrics)

        with open(self.log_file, "a", encoding="utf-8") as f:
            f.write(f"Epoch {state.epoch} evaluation metrics:\n")
            for k, v in metrics.items():
                f.write(f"{k}: {v}\n")
            f.write("\n")


training_args = TrainingArguments(
    output_dir="./tuned",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=2000,
    weight_decay=weight_decay,
    logging_dir="./logs",
    logging_steps=50,
    evaluation_strategy="epoch",
    save_total_limit=10,
    save_strategy="epoch",
    report_to=["wandb"],
)


trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


trainer.add_callback(MetricsLoggerCallback(log_file="training_log.txt"))

wandb.watch(model, log="all")

trainer.train()

model.save_pretrained(os.path.join(training_args.output_dir, "final_model"))
tokenizer.save_pretrained(os.path.join(training_args.output_dir, "final_model"))

wandb.finish()


In [12]:
%pip install pipreqs --break-system-packages

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=20d981ee55213915d46090935e8876f2db33643b3d32d0b7a6b62486fc541407
  Stored in directory: /Users/valexv/Library/Caches/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 8.28.0
    Uninstalling ipython-8.28.0:
      Successfully uninstalled ipython-8.28.0

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.12 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
!pipreqs ./ --force

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


INFO: Not scanning for jupyter notebooks.
INFO: Successfully saved requirements file in ./requirements.txt


In [18]:
dataset['train'][:10]

{'text': ['Приложение \n\nк приказу НИУ ВШЭ \nот 30.07.2021 № 6.18.1-01/300721-5\n\nУТВЕРЖДЕН\nученым советом ФЭН\nпротокол № 13 от 29.06.2021\n\n\n\n\nПорядок назначения и выплаты\nстипендий имени Л.Л. Любимова факультета экономических наук \nНационального исследовательского университета «Высшая школа экономики» для студентов и аспирантов, участвующих в долгосрочных программах международной академической мобильности и программах двух дипломов\n\n\nОбщие положения\nПорядок назначения и выплаты стипендий имени Л.Л. Любимова  факультета экономических наук Национального исследовательского университета «Высшая школа экономики» для студентов и аспирантов, участвующих в долгосрочных программах международной академической мобильности и программах двух дипломов (далее соответственно – Порядок, Стипендия, ФЭН, НИУ ВШЭ) определяет критерии отбора получателей, сроки, порядок назначения и выплаты Стипендии для студентов и аспирантов, участвующих в долгосрочных программах международной академическо